## Sequential usage

This notebook explains the basic usage of `parafields`. Everything that is explained here also holds true for the parallel case.

In [ ]:
import parafields
import numpy as np

The main entry point for the generation of Gaussian random fields is the `generate_field` function:

In [ ]:
field = parafields.generate_field(
    cells=(256, 256), extensions=(1.0, 1.0), covariance="exponential", variance=1.0
)

The `cells` parameter defines the resolution of the random field and can be a tuple of length 1, 2 or 3. The `extensions` parameter defines the size of the domain that the field is defined on. The `covariance` and `variance` define those stochastic properties of the field. The resulting variable `field` is of type `parafields.RandomField` and can easily be visualized in a Jupyter notebook:

In [ ]:
field

In order to use the random field in your application, you can evaluate it on the entire domain, yielding a `d`-dimensional `numpy` array for further processing:

In [ ]:
values = field.evaluate()

In [ ]:
values

### More stochastic properties

The `generate_field` function supports a lot more parameter that control stochastic properties of the field. For a full reference, you should see the [Full API documentation](https://parafields.readthedocs.io/en/latest/api.html#parafields.generate_field). Here, we show some illustrative examples:

In [ ]:
parafields.generate_field(cells=(256, 256), covariance="cubic", variance=1.0)

In [ ]:
parafields.generate_field(
    cells=(256, 256), covariance="cauchy", variance=0.5, corrLength=0.25
)

In [ ]:
parafields.generate_field(cells=(256, 256), covariance="whiteNoise", variance=0.1)

In [ ]:
parafields.generate_field(
    cells=(256, 256),
    covariance="gaussian",
    variance=0.1,
    corrLength=0.25,
    periodic=True,
)

In [ ]:
parafields.generate_field(
    cells=(256, 256),
    covariance="exponential",
    variance=1.0,
    anisotropy="axiparallel",
    corrLength=[0.05, 0.2],
)

### Custom covariance functions

`parafields` also allows the definition of user-defined stochastic input in Python. These functions are then called directly from the C++ backend. In this example, we redefine the exponential covariance structure that is available with `covariance="exponential"`:

In [ ]:
def exponential(variance, x):
    return variance * np.exp(-np.linalg.norm(x))

In [ ]:
parafields.generate_field(cells=(256, 256), covariance=exponential, variance=0.1)

This is a very flexible tool for method development and rapid prototyping, but you should carefully look at the performance implications of this approach if using it in production (e.g. above example is slower by a factor of ~20).

### Trend components

So far, we only generated the stochastic part of a random field. However, `parafields` can also generate a variety of trend components that use the same random number generator. These are added by calling the respective methods on the field object:

In [ ]:
field = parafields.generate_field(cells=(256, 256), covariance="exponential")
field.add_mean_trend_component(mean=0.5, variance=0.1)

In [ ]:
field = parafields.generate_field(cells=(256, 256), covariance="exponential")
field.add_slope_trend_component(mean=[0.5, 0.2], variance=[0.1, 0.01])

In [ ]:
field = parafields.generate_field(cells=(256, 256), covariance="exponential")
field.add_disk_trend_component(
    mean_position=[0.25, 0.25],
    variance_position=[0.1, 0.1],
    mean_radius=0.1,
    variance_radius=0.01,
    mean_height=3.0,
    variance_height=0.1,
)

In [ ]:
field = parafields.generate_field(cells=(256, 256), covariance="exponential")
field.add_block_trend_component(
    mean_position=[0.25, 0.25],
    variance_position=[0.1, 0.1],
    mean_extent=[0.2, 0.2],
    variance_extent=[0.01, 0.01],
    mean_height=1.0,
    variance_height=0.1,
)

### Deterministic Field generation

The pseudo-random number generator that is used by `parafields` can be provided a seed. By default, the `seed` is parameter is `None`, which means that a new seed will be generated on each run. However, the seed can be explicitly set in order to allow deterministic field generation:

In [ ]:
field1 = parafields.generate_field(
    cells=(256, 256), covariance="exponential", variance=1.0, seed=42
)

In [ ]:
field2 = parafields.generate_field(
    cells=(256, 256), covariance="exponential", variance=1.0, seed=42
)

In [ ]:
np.allclose(field1.evaluate(), field2.evaluate())

If you want to create new realization of a random field with another seed, you can also explicitly regenerate the field:

In [ ]:
field1.generate(seed=0)